# COVID-19-BERT-ResearchPapers-Semantic-Search

This work builds a **semantic search engine using BERT**, to search a query through the dataset of research papers provided as part of [Kaggle's competion CORD-19-research-challenge](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge), we like to thank kaggle and all of the competion sponsers for this competion in bringing up efforts for fighting this virus.

This work, 
1.   first divides the dataset to paragraphs
2.   then uses BERT to embedded paragraphs of papers using bert-base-nli-mean-tokens pretrained model
3.   finally runs a query and returns the top 5 paragraphs and their papers' titles,abstract,abstract_summary

We have built this notebook to run seamlesly on google colab, connect with google drive, and downalod the data using [kaggle api](https://github.com/Kaggle/kaggle-api), so no data is downloaded to your device, and no need to have a powerful GPU, as all is done freely through google colab, we like to thank google for providing the research community with google colab

**Code** is found here [on github ](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search), we truly hope that this work has a postive impact in the fight aganist this evil virus, we truly pray for all people to be able to win this fight.


**References** :

*   We use the library provided by [UKPLab](https://github.com/UKPLab) called [sentence-transformers](https://github.com/UKPLab/sentence-transformers), this library makes it truly easy to use BERT and other architectures like ALBERT,XLNet for sentence embedding, they also provide simple interface to query and cluster data.
*   We have used the code from [maksimeren](https://www.kaggle.com/maksimeren/covid-19-literature-clustering) for data processing, we truly like to thank him.
*   We used the concept of drawing BERT, disccussed here [Jay Alammar](http://jalammar.github.io/) in illustrating how our architecture works, his blogs are extremly informative and easily understood.
*   We used the pre-trained models disccess in Conneau et al., 2017, show in the InferSent-Paper (Supervised Learning of Universal Sentence Representations from Natural Language Inference Data) that training on Natural Language Inference (NLI) data can produce universal sentence embeddings.






## Architecture

The paper is found in the json from the dataset in paragraphs, we use this division in the papers, then we pass these paragraphs to a pre-trained BERT model [bert-base-nli-mean-tokens](https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md) to be embedded


![alt text](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search/blob/master/assets/Bert%20Information%20Retrival_Train.jpg?raw=true)


After embedding is done, we pass the embedded the query using the same bert model

Then we compare the both embedding represnetations (paragraphs and query) using cosine similarity, we then return the most similar paragraphs with their paper details (title,abstract,abstract_summary)

![alt text](https://github.com/theamrzaki/COVID-19-BERT-ResearchPapers-Semantic-Search/blob/master/assets/Bert%20Information%20Retrival_Test.jpg?raw=true)

In [0]:
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers

In [0]:
#install the kaggle data to google colab
#https://github.com/Kaggle/kaggle-api#api-credentials
!pip install kaggle
import os
!cp "/content/kaggle.json" /root/.kaggle
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge
!unzip  CORD-19-research-challenge.zip -d /content/CORD-19-research-challenge

## Data Processing

built using
https://www.kaggle.com/maksimeren/covid-19-literature-clustering

In [0]:
import glob
import json
import pandas as pd
from tqdm import tqdm
root_path = '/content/CORD-19-research-challenge/'
all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

In [0]:
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

### Read Data (Helpers)

In [0]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

In [0]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [0]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

### Handle Possible Duplicates

In [0]:
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid['abstract'].describe(include='all')

In [0]:
df_covid['body_text'].describe(include='all')

### Take a Look at the Data

In [0]:
df_covid.head()

In [0]:
df_covid.describe()

### Data Pre-Process

In [0]:
df_covid.dropna(inplace=True)
df_covid.info()

In [0]:
df_covid = df_covid.head(12500)

In [0]:
import re

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [0]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: lower_case(x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: lower_case(x))

In [0]:
df_covid.head(4)

In [0]:
df_covid.to_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid.csv")

In [0]:
df_covid_test = pd.read_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid.csv")
text = df_covid_test.drop(["authors", "journal", "Unnamed: 0"], axis=1)
text.head(5)

In [0]:
text_dict = text.to_dict()
len_text = len(text_dict["paper_id"])

In [0]:
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences.csv")
df_sentences.head()

In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences_Full.csv")
df_sentences.head()

## Preparing Data for Embedding

In [0]:
import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")

In [0]:
df_sentences.head()

In [0]:
df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

In [0]:
list(df_sentences.keys())[:5]

In [0]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/BertSentenceSimilarity/Data/covid_sentences_Full.csv", index_col=0)
df.head()

## BERT

In [0]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

# Corpus with example sentences
corpus = df_sentences_list
#corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)
with open("/content/drive/My Drive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "rb") as file_:
  corpus_embeddings = pkl.load(file_)

# Query sentences:
queries = ['What has been published about medical care?',
           'Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest',
           'Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually',
           'Resources to support skilled nursing facilities and long term care facilities.',
           'Mobilization of surge medical staff to address shortages in overwhelmed communities .',
           'Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies .']
query_embeddings = embedder.encode(queries,show_progress_bar=True)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")

Batches: 100%|██████████| 1/1 [00:00<00:00, 27.80it/s]

Top 5 most similar sentences in corpus:


=========================================================
==========================Query==============================
=== What has been published about medical care? =====
=========================================================
Score:    (Score: 0.8296) 

Paragraph:    how may state authorities require persons to undergo medical treatment 

paper_id:   1950c30fea7ef227129d94831df3fd0c57b9802c 

Title:    Chapter 10 Legal Aspects of Biosecurity 

Abstract:   when bad men combine the good must associate else they will fall one by one an unpitied sacrifice in a contemptible struggle
the study of this chapter will enable you to
1 discuss the definitions of terrorism and weapons of mass destruction and their relation to the illicit use of biological agents
2 list all legislative and administrative documents that address the legal aspects of the unlawful use of biological agents
4 discuss the prohibited uses of biological agents under international law 5 list and briefly discuss the homeland security presidential directives that apply to biosecurity and biodefense 

Abstract_Summary:    When bad men combine, the good must associate;<br>else they will fall one by one, an unpitied sacrifice<br>in a contemptible struggle.
The study of this<br>chapter will enable you to:
1. Discuss the definitions<br>of terrorism and weapons of mass destruction and<br>their relation to the illicit use of biological<br>agents.
2. List all legislative and administrative<br>documents that address the legal aspects of the unlawful<br>use of biological agents.
4. Discuss the<br>prohibited uses of biological agents under international<br>law. 5. List and briefly discuss the Homeland<br>Security Presidential Directives that apply to<br>biosecurity and biodefense. 

-------------------------------------------
Score:    (Score: 0.8220) 

Paragraph:    to identify how one health has been used recently in the medical literature 

paper_id:   f703b510de361a759c9fe3419fa3aeb092c95512 

Title:    One Health and Zoonoses: The Evolution of One<br>Health and Incorporation of Zoonoses 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.8187) 

Paragraph:    medical questions in the personal questionnaire included the following 

paper_id:   2afa3da371e5495dd55f0b4dde4ae7700eaf99d6 

Title:    Rhinitis, Asthma and Respiratory Infections<br>among Adults in Relation to the Home Environment in<br>Multi-Family Buildings in Sweden 

Abstract:   risk factors for rhinitis asthma and respiratory infections in the home environment were studied by a questionnaire survey totally 5775 occupants 18 years old from a stratified random sample of multifamily buildings in sweden participated 46 510 had rhinitis in the last 3 months current rhinitis 115 doctor diagnosed asthma 464 respiratory infections in the last 3 months and 119 antibiotic medication for respiratory infections in the last 12 months associations between home environment and health were analyzed by multiple logistic regression controlling for gender age and smoking and mutual adjustment buildings constructed during 19601975 were risk factors for day time breathlessness or  153 95ci 103229 and those constructed during 19761985 had more current rhinitis or  143 95ci 112184 and respiratory infections or  146 95ci 121178 cities with higher population density had more current rhinitis p  0008 and respiratory infections p0001 rented apartments had more current rhinitis or  123 95ci 107140 wheeze or  120 95ci 102141 day time breathlessness or  131 95ci 104166 and respiratory infections or  113 95ci 101126 living in colder parts of the country was a risk factor for wheeze p  003 and night time breathlessness p  0002 building dampness was a risk factor for wheeze or  142 95ci 108186 and day time breathlessness or  157 95ci 109227 building dampness was a risk factor for health among those below 66 years old odor at home was a risk factor for doctor diagnosed asthma or  149 95ci 108206 and current asthma or  152 95ci 103224 environmental tobacco smoke ets was a risk factor for current asthma or  153 95ci 109216 window pane condensation was a risk factor for antibiotic medication for respiratory infections or  141 95ci 110182 in conclusion rhinitis asthma and respiratory infections were related to a number of factors in the home environment certain building years  building dampness window pane condensation and odor in the dwelling may be risk factors 

Abstract_Summary:    Risk factors for rhinitis, asthma and<br>respiratory infections in the home environment were<br>studied by a questionnaire survey. Totally 5775<br>occupants ($18 years old) from a stratified random sample<br>of multi-family buildings in Sweden<br>participated (46%). 51.0% had rhinitis in the last 3 months<br>(current rhinitis); 11.5% doctor diagnosed asthma;<br>46.4% respiratory infections in the last 3 months and<br>11.9% antibiotic medication for respiratory<br>infections in the last 12 months. Associations between<br>home environment and health were analyzed by<br>multiple logistic regression, controlling for gender,<br>age and smoking and mutual adjustment. Buildings<br>constructed during 1960-1975 were risk factors for day... 

-------------------------------------------
Score:    (Score: 0.7816) 

Paragraph:    when you spoke to nhs direct what did they tell you 

paper_id:   6fc8c1b4cfbd790cbe02c825fd6f9181b1fc6645 

Title:    Reassuring and managing patients with<br>concerns about swine flu: Qualitative interviews with<br>callers to NHS Direct 

Abstract:   background during the early stages of the 2009 swine flu influenza h1n1 outbreak the large majority of patients who contacted the health services about the illness did not have it in the uk the nhs direct telephone service was used by many of these patients we used qualitative interviews to identify the main reasons why people approached nhs direct with concerns about swine flu and to identify aspects of their contact which were reassuring using a framework approach methods 33 patients participated in semistructured interviews all patients had telephoned nhs direct between 11 and 14 may with concerns about swine flu and had been assessed as being unlikely to have the illness results reasons for seeking advice about swine flu included the presence of unexpectedly severe flulike symptoms uncertainties about how one can catch swine flu concern about giving it to others pressure from friends or employers and seeking peace of mind most participants found speaking to nhs direct reassuring or useful helpful aspects included having swine flu ruled out receiving an alternative explanation for symptoms clarification on how swine flu is transmitted and the perceived credibility of nhs direct noone reported anything that had increased their anxiety and only one participant subsequently sought additional advice about swine flu from elsewhere conclusions future major incidents involving other forms of chemical biological or radiological hazards may also cause large numbers of unexposed people to seek health advice our data suggest that providing telephone triage and information is helpful in such instances particularly where advice can be given via a trusted preexisting service 

Abstract_Summary:    Background: During the early stages of the 2009<br>swine flu (influenza H1N1) outbreak, the large<br>majority of patients who contacted the health services<br>about the illness did not have it. In the UK, the NHS<br>Direct telephone service was used by many of these<br>patients. We used qualitative interviews to identify the<br>main reasons why people approached NHS Direct with<br>concerns about swine flu and to identify aspects of their<br>contact which were reassuring, using a framework<br>approach. Methods: 33 patients participated in<br>semi-structured interviews. All patients had telephoned NHS<br>Direct between 11 and 14 May with concerns about swine... 

-------------------------------------------
Score:    (Score: 0.7758) 

Paragraph:    how should this patient be evaluated and treated 

paper_id:   31881cc2530107849da340e6a16c9618b7e7bb8d 

Title:    Hemophagocytic lymphohistiocytosis<br>associated with viral infections: Diagnostic challenge<br>and therapeutic dilemma 

Abstract:   hemophagocytic lymphohistiocytosis is a frequently fatal clinicopathologic syndrome in which an uncontrolled and ineffective immune response leads to severe hyperinflammation it may occur as either a familial disorder or a sporadic condition in association with a variety of triggers infections malignancies autoimmune diseases and acquired immune deficiencies however the most consistent association is with viral infections especially epsteinbarr virus the main clinical features are fever liver dysfunction coagulation abnormalities and pancytopenia early diagnosis and treatment are important to reducing mortality but the diagnosis is difficult because of the rarity of the syndrome and the lack of specificity of the clinical findings treatment should be directed toward treating the underlying disease and to suppressing the exaggerated inflammatory response through the use of immunosuppressive agents
resumen la linfohistiocitosis hemofagoctica es un sndrome clinicopatolgico de evolucin potencialmente fatal en el que una respuesta inmune no controlada e ineficaz conduce a hiperinflamacin puede aparecer como una enfermedad familiar o espordica asociado a diferentes factores desencadenantes infecciones neoplasias enfermedades autoinmunes o inmunodeficiencias adquiridas pero la asociacin ms consistente es con infecciones virales  please cite this article as mostazafernndez jl guerra laso j carriedo ule d ruiz de morales jmg linfohistiocitosis hemofagoctica asociada a infecciones virales reto diagnstico y dilema teraputico rev clin esp 2014214320327 

Abstract_Summary:    Hemophagocytic lymphohistiocytosis is a<br>frequently fatal clinicopathologic syndrome in which an<br>uncontrolled and ineffective immune response leads to<br>severe hyperinflammation. It may occur as either a<br>familial disorder or a sporadic condition in<br>association with a variety of triggers: infections,<br>malignancies, autoimmune diseases, and acquired immune<br>deficiencies. However, the most consistent association is<br>with viral infections, especially Epstein---Barr<br>virus. The main clinical features are fever, liver<br>dysfunction, coagulation abnormalities and pancytopenia.<br>Early diagnosis and treatment are important to<br>reducing mortality, but the diagnosis is difficult<br>because of the rarity of the syndrome and the lack of<br>specificity of the clinical... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest =====
=========================================================
Score:    (Score: 0.8139) 

Paragraph:    clinical signs in hcm are explained by leftsided chf complications of arterial thromboembolism ate lv outflow tract obstruction or arrhythmias capable of causing syncope or sudden cardiac death 

paper_id:   1fdc1a513291d7613c95bbefae5b7ea0e811d5a6 

Title:    Chapter 150 Cardiomyopathy 

Abstract:   1527
1528 section 11  cardiopulmonary system this chapter will next describe the clinical features of feline cardiomyopathies and the therapy of related complications following this is a consideration of canine dcm and arrhythmogenic cardiomyopathy
overview and pathophysiology of feline hcm  feline idiopathic hcm is characterized by hypertrophy and thickening of the left ventricle unexplained by congenital heart disease systemic hypertension or an endocrinopathy
 the condition is genetic in a number of feline breeds including the maine coon cat persian cat and the ragdoll thus far one sarcomeric mutation has been identified 

Abstract_Summary:    1527
1528 Section 11 / Cardiopulmonary System<br>This chapter will next describe the clinical<br>features of feline cardiomyopathies and the therapy of<br>related complications. Following this is a<br>consideration of canine DCM and arrhythmogenic<br>cardiomyopathy.
Overview and Pathophysiology of Feline HCM • Feline<br>idiopathic HCM is characterized by hypertrophy and<br>thickening of the left ventricle unexplained by<br>congenital heart disease, systemic hypertension, or an<br>endocrinopathy.
• The condition is genetic in a number of feline<br>breeds, including the Maine coon cat, Persian cat, and<br>the Ragdoll. Thus far, one sarcomeric mutation has<br>been identified. 

-------------------------------------------
Score:    (Score: 0.7966) 

Paragraph:    the term arrhythmogenic cardiomyopathy is a useful expression that refers to recurrent or persistent ventricular or atrial arrhythmias in the setting of a normal echocardiogram the most commonly observed rhythm disturbances are pvcs and ventricular tachycardia vt however atrial rhythm disturbances may be recognized including atrial fibrillation paroxysmal or sustained atrial tachycardia and atrial flutter 

paper_id:   1fdc1a513291d7613c95bbefae5b7ea0e811d5a6 

Title:    Chapter 150 Cardiomyopathy 

Abstract:   1527
1528 section 11  cardiopulmonary system this chapter will next describe the clinical features of feline cardiomyopathies and the therapy of related complications following this is a consideration of canine dcm and arrhythmogenic cardiomyopathy
overview and pathophysiology of feline hcm  feline idiopathic hcm is characterized by hypertrophy and thickening of the left ventricle unexplained by congenital heart disease systemic hypertension or an endocrinopathy
 the condition is genetic in a number of feline breeds including the maine coon cat persian cat and the ragdoll thus far one sarcomeric mutation has been identified 

Abstract_Summary:    1527
1528 Section 11 / Cardiopulmonary System<br>This chapter will next describe the clinical<br>features of feline cardiomyopathies and the therapy of<br>related complications. Following this is a<br>consideration of canine DCM and arrhythmogenic<br>cardiomyopathy.
Overview and Pathophysiology of Feline HCM • Feline<br>idiopathic HCM is characterized by hypertrophy and<br>thickening of the left ventricle unexplained by<br>congenital heart disease, systemic hypertension, or an<br>endocrinopathy.
• The condition is genetic in a number of feline<br>breeds, including the Maine coon cat, Persian cat, and<br>the Ragdoll. Thus far, one sarcomeric mutation has<br>been identified. 

-------------------------------------------
Score:    (Score: 0.7873) 

Paragraph:    many cv diseases are tightly associated with inflammatory immune responses in their pathogenesis this include a wide range of vascular diseases endotoxin shock acute vasculitis atherosclerosis postoperative stenosis allograft vasculopathy cardiac diseases cardiac hypertrophy dilated cardiomyopathy myocardial infarction or ischemiareperfusion injury and myocarditis [133] [134] [135] [136] [137] [138] [139] [140] [141] [142]  and several autoimmune diseases in the cv system [8]  in the final section of this review we will briefly overview what is presently known about some of these cv diseases in terms of trp channel physiology and pathophysiology see also table 1 

paper_id:   9221c3ed344b506c1208f8c2c4f9bf31f60b89da 

Title:    “TRP inflammation” relationship in<br>cardiovascular system 

Abstract:   despite considerable advances in the research and treatment the precise relationship between inflammation and cardiovascular cv disease remains incompletely understood therefore understanding the immunoinflammatory processes underlying the initiation progression and exacerbation of many cardiovascular diseases is of prime importance the innate immune system has an ancient origin and is well conserved across species its activation occurs in response to pathogens or tissue injury recent studies suggest that altered ionic balance and production of noxious gaseous mediators link to immune and inflammatory responses with altered ion channel expression and function among plausible candidates for this are transient receptor potential trp channels that function as polymodal sensors and scaffolding proteins involved in many physiological and pathological processes in this review we will first focus on the relevance of trp channel to both exogenous and endogenous factors related to innate immune response and transcription factors related to sustained inflammatory status the emerging role of inflammasome to regulate innate immunity and its possible connection to trp channels will also be discussed secondly we will discuss about the linkage of trp channels to inflammatory cv diseases from a viewpoint of inflammation in a general sense which is not restricted to the innate immunity these knowledge may serve to provide new insights into the pathogenesis of various inflammatory cv diseases and their novel therapeutic strategies 

Abstract_Summary:    Despite considerable advances in the research<br>and treatment, the precise relationship between<br>inflammation and cardiovascular (CV) disease remains<br>incompletely understood. Therefore, understanding the<br>immunoinflammatory processes underlying the initiation,<br>progression, and exacerbation of many cardiovascular<br>diseases is of prime importance. The innate immune<br>system has an ancient origin and is well conserved<br>across species. Its activation occurs in response to<br>pathogens or tissue injury. Recent studies suggest that<br>altered ionic balance, and production of noxious<br>gaseous mediators link to immune and inflammatory<br>responses with altered ion channel expression and<br>function. Among plausible candidates for this are<br>transient receptor potential (TRP) channels... 

-------------------------------------------
Score:    (Score: 0.7791) 

Paragraph:    cardiac ppar ppar and ppar1 have pivotal roles in the pathophysiology of diabetic cardiomyopathy [44] 

paper_id:   f29df5233e688dddd0286374e0993e43459fe04b 

Title:    MuRF2 regulates PPARγ1 activity to protect<br>against diabetic cardiomyopathy and enhance weight<br>gain induced by a high fat diet 

Abstract:   background in diabetes mellitus the morbidity and mortality of cardiovascular disease is increased and represents an important independent mechanism by which heart disease is exacerbated the pathogenesis of diabetic cardiomyopathy involves the enhanced activation of ppar transcription factors including ppar and to a lesser degree ppar and ppar1 how these transcription factors are regulated in the heart is largely unknown recent studies have described posttranslational ubiquitination of ppars as ways in which ppar activity is inhibited in cancer however specific mechanisms in the heart have not previously been described recent studies have implicated the musclespecific ubiquitin ligase muscle ring finger2 murf2 in inhibiting the nuclear transcription factor srf initial studies of murf2 hearts revealed enhanced ppar activity leading to the hypothesis that murf2 regulates ppar activity by posttranslational ubiquitination
methods murf2 mice were challenged with a 26week 60 fat diet designed to simulate obesitymediated insulin resistance and diabetic cardiomyopathy mice were followed by conscious echocardiography blood glucose tissue triglyceride glycogen levels immunoblot analysis of intracellular signaling heart and skeletal muscle morphometrics and ppar ppar and ppar1regulated mrna expression 

Abstract_Summary:    Background: In diabetes mellitus the<br>morbidity and mortality of cardiovascular disease is<br>increased and represents an important independent<br>mechanism by which heart disease is exacerbated. The<br>pathogenesis of diabetic cardiomyopathy involves the<br>enhanced activation of PPAR transcription factors,<br>including PPARα, and to a lesser degree PPARβ and PPARγ1.<br>How these transcription factors are regulated in<br>the heart is largely unknown. Recent studies have<br>described post-translational ubiquitination of PPARs<br>as ways in which PPAR activity is inhibited in<br>cancer. However, specific mechanisms in the heart have<br>not previously been described. Recent studies<br>have implicated the musclespecific ubiquitin<br>ligase muscle ring finger-2... 

-------------------------------------------
Score:    (Score: 0.7782) 

Paragraph:    currently symmetrical peripheral gangrene in sepsis is suspected to occur due to vasopressor agents used in septic shock however a better explanation rests on combined hemostasis of microthrombogenesis and macrothrombogenesis although the latter is secondary event occurring as a complication of eavmtddit the pathophysiological mechanism of combined micromacrothrombotic syndrome can be easily supported by twopath unifying theory typically first setup is the hospital admission of a patient with sepsis that provokes eavmtddit the patient would need multiple venous and arterial accesses andor perhaps surgery during hospitalization [91]  this vascular intervention causes intravascular injury leading to damage of setevt and release of tf activated tf path leads to fibrinogenesis abbreviations apl acute promyelocytic leukemia dic disseminated intravascular coagulation dic illfounded dic dit disseminated intravascular microthrombosis fhf fulminant hepatic failure maha microangiopathic hemolytic anemia mods multiorgan dysfunction syndrome pt prothrombin time aptt activated partial thromboplastin time ttp thrombotic thrombocytopenic purpura ulvwf unusually large von willebrand factor multimers eavmtd endotheliopathyassociated vascular microthrombotic disease a please note that dic without hepatic coagulopathy is exactly the same to eavmtddit without hepatic coagulopathy which is also consistent with the thesis that dic is dit and forms fibrin clots in circulation since ongoing ulvwf path is already active from endotheliopathy and microthrombi strings are being formed fibrin clots and some of microthrombi would be unified and produce intravascular macrothrombi via macrothrombogenesis [20]  macrothrombi could travel to the peripheral arterial vasculatures in small and large arteries of the digits and extremities this combined micromacrothrombotic syndrome could lead to multiple welldemarcated peripheral symmetrical gangrene at distal vascular trees 

paper_id:   879b563cbe739c7ec914ce9eef39d9a85d6d3801 

Title:    Sepsis and septic shock: endothelial<br>molecular pathogenesis associated with vascular<br>microthrombotic disease 

Abstract:   in addition to protective  

Abstract_Summary:    In addition to protective " 

-------------------------------------------


=========================================================
==========================Query==============================
=== Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually =====
=========================================================
Score:    (Score: 0.8002) 

Paragraph:    conclusion several methods and approaches could be used in the healthcare arena time series is an analytical tool to study diseases and resources management at healthcare institutions the flexibility to follow up and recognize data patterns and provide explanations must not be neglected in studies of healthcare interventions in this study the arima model was introduced without the use of mathematical details or other extensions to the model the investigator or the healthcare organization involved in disease management programs could have great advantages when using analytical methodology in several areas with the ability to perform provisions in many cases despite the analytical possibility by statistical means this approach does not replace investigators common sense and experience in disease interventions 

paper_id:   1b8a085682d10dad4baf90417a8cbcde144cc8c9 

Title:    Disease management with ARIMA model in time<br>series 

Abstract:   the evaluation of infectious and noninfectious disease management can be done through the use of a time series analysis in this study we expect to measure the results and prevent intervention effects on the disease clinical studies have benefited from the use of these techniques particularly for the wide applicability of the arima model this study briefly presents the process of using the arima model this analytical tool offers a great contribution for researchers and healthcare managers in the evaluation of healthcare interventions in specific populations 

Abstract_Summary:    The evaluation of infectious and<br>noninfectious disease management can be done through the use<br>of a time series analysis. In this study, we expect<br>to measure the results and prevent intervention<br>effects on the disease. Clinical studies have<br>benefited from the use of these techniques, particularly<br>for the wide applicability of the ARIMA model. This<br>study briefly presents the process of using the ARIMA<br>model. This analytical tool offers a great<br>contribution for researchers and healthcare managers in the<br>evaluation of healthcare interventions in specific<br>populations. 

-------------------------------------------
Score:    (Score: 0.7745) 

Paragraph:    whether the health sector is in fact more skillintensive than all other sectors is an empirical question as is that of whether the incidence of illness and the provision and effectiveness of health care are independent of labour type in a multisectoral model with more than two factors possibly health carespecific and other reallife complexities the foregoing predictions are unlikely to be wholly true nevertheless these effects will still operate in the background and thus give a useful guide to the interpretation of the outcomes of such a model 

paper_id:   c33e8d913713308e554815bda48d2bfc618f6943 

Title:    A comparative analysis of some policy options<br>to reduce rationing in the UK's NHS: Lessons from a<br>general equilibrium model incorporating positive<br>health effects 

Abstract:   this paper seeks to determine the macroeconomic impacts of changes in health care provision the resource allocation issues have been explored in theory by applying the rybczynski theorem and empirically using a computable general equilibrium cge model for the uk with a detailed health component from the theory changes in nonhealth outputs are shown to depend on factorbias and scale effects the net effects generally being indeterminate from the applied model a rise in the national health service nhs budget is shown to yield overall welfare gains which fall by twothirds assuming health carespecific factors a nominally equivalent migration policy yields even higher welfare gains 

Abstract_Summary:    This paper seeks to determine the<br>macro-economic impacts of changes in health care provision.<br>The resource allocation issues have been explored<br>in theory, by applying the Rybczynski theorem,<br>and empirically, using a computable general<br>equilibrium (CGE) model for the UK with a detailed health<br>component. From the theory, changes in non-health outputs<br>are shown to depend on factor-bias and scale<br>effects, the net effects generally being<br>indeterminate. From the applied model, a rise in the National<br>Health Service (NHS) budget is shown to yield overall<br>welfare gains, which fall by two-thirds assuming<br>health care-specific factors. A nominally<br>equivalent migration policy... 

-------------------------------------------
Score:    (Score: 0.7576) 

Paragraph:    the central question is the following how can triage systems in emergency care be ethically assessed so as to realize optimal use of scarce resources in an ethically just way without remaining on the abstract level that is by taking the effect of triage on the individual patients and caregivers into account 

paper_id:   34190b8df3ed825e3c49e41a3782d04acf1e21e9 

Title:    Emergency department triage: an ethical<br>analysis 

Abstract:   background emergency departments across the globe follow a triage system in order to cope with overcrowding the intention behind triage is to improve the emergency care and to prioritize cases in terms of clinical urgency
discussion in emergency department triage medical care might lead to adverse consequences like delay in providing care compromise in privacy and confidentiality poor physicianpatient communication failing to provide the necessary care altogether or even having to decide whose life to save when not everyone can be saved these consequences challenge the ethical quality of emergency care this article provides an ethical analysis of routine emergency department triage the four principles of biomedical ethics viz respect for autonomy beneficence nonmaleficence and justice provide the starting point and help us to identify the ethical challenges of emergency department triage however they do not offer a comprehensive ethical view to address the ethical issues of emergency department triage from a more comprehensive ethical view the care ethics perspective offers additional insights summary we integrate the results from the analysis using four principles of biomedical ethics into care ethics perspective on triage and propose an integrated clinically and ethically based framework of emergency department triage planning as seen from a comprehensive ethics perspective that incorporates both the principlesbased and careoriented approach 

Abstract_Summary:    Background: Emergency departments across the<br>globe follow a triage system in order to cope with<br>overcrowding. The intention behind triage is to improve the<br>emergency care and to prioritize cases in terms of<br>clinical urgency.
Discussion: In emergency<br>department triage, medical care might lead to adverse<br>consequences like delay in providing care, compromise in<br>privacy and confidentiality, poor physician-patient<br>communication, failing to provide the necessary care<br>altogether, or even having to decide whose life to save when<br>not everyone can be saved. These consequences<br>challenge the ethical quality of emergency care. This<br>article provides an ethical analysis of "routine"<br>emergency department... 

-------------------------------------------
Score:    (Score: 0.7564) 

Paragraph:    limitations of the study our findings are exploratory rather than definitive and indicate directions for further planning and research like any new tool the framework and its application in a given context needs testing and refinement through simulation exercises targeting ambulatory care services as well as the broader health system 

paper_id:   646484c8081b3f1bf336bae89411c2aa86885788 

Title:    General Practice and Pandemic Influenza: A<br>Framework for Planning and Comparison of Plans in Five<br>Countries 

Abstract:   background although primary health care and in particular general practice will be at the frontline in the response to pandemic influenza there are no frameworks to guide systematic planning for this task or to appraise available plans for their relevance to general practice we aimed to develop a framework that will facilitate planning for general practice and used it to appraise pandemic plans from australia england usa new zealand and canada
methodologyprincipal findings we adapted the haddon matrix to develop the framework populating its cells through a multimethod study that incorporated the peerreviewed and grey literature interviews with general practitioners practice nurses and senior decisionmakers and desktop simulation exercises we used the framework to analyse 89 publiclyavailable jurisdictional plans at similar managerial levels in the five countries the framework identifies four functional domains clinical care for influenza and other needs public health responsibilities the internal environment and the macroenvironment of general practice no plan addressed all four domains most plans either ignored or were sketchy about noninfluenza clinical needs and about the contribution of general practice to public health beyond surveillance collaborations between general practices were addressed in few plans and interrelationships with the broader health system even less frequently
conclusions this is the first study to provide a framework to guide general practice planning for pandemic influenza the framework helped identify critical shortcomings in available plans engaging general practice effectively in planning is challenging particularly where governance structures for primary health care are weak we identify implications for practice and for research 

Abstract_Summary:    Background: Although primary health care, and<br>in particular, general practice will be at the<br>frontline in the response to pandemic influenza, there<br>are no frameworks to guide systematic planning for<br>this task or to appraise available plans for their<br>relevance to general practice. We aimed to develop a<br>framework that will facilitate planning for general<br>practice, and used it to appraise pandemic plans from<br>Australia, England, USA, New Zealand and<br>Canada.
Methodology/Principal Findings: We adapted the Haddon matrix to<br>develop the framework, populating its cells through a<br>multi-method study that incorporated the peer-reviewed and<br>grey literature, interviews with general<br>practitioners, practice... 

-------------------------------------------
Score:    (Score: 0.7471) 

Paragraph:    cea measures have and can be used to help resource allocation decisions without formally using a specific cutoff the common use of league tables to help people understand where a medical intervention or technology fits in the general realm of common use and practice can inform some decisions about health technology use in these settings an implicit cea cutoff is used in lieu of an explicit measure this approach however tacitly assumes that existing interventions in common use have passed an implicit or explicit costeffectiveness test which is not necessarily true 

paper_id:   5719f6793a83d07462521f608b01fe66efeb6500 

Title:    Approaches to Aggregation and Decision<br>Making—A Health Economics Approach: An ISPOR Special<br>Task Force Report [5] 

Abstract:   the fifth section of our special task force report identifies and discusses two aggregation issues 1 aggregation of cost and benefit information across individuals to a population level for benefit plan decision making and 2 combining multiple elements of value into a single value metric for individuals first we argue that additional elements could be included in measures of value but such elements have not generally been included in measures of qualityadjusted lifeyears for example we describe a recently developed extended costeffectiveness analysis ecea that provides a good example of how to use a broader concept of utility ecea adds two featuresmeasures of financial risk protection and income distributional consequences we then discuss a further option for expanding this approachaugmented cea which can introduce many value measures neither of these approaches however provide a comprehensive measure of value to resolve this issue we review a technique called multicriteria decision analysis that can provide a comprehensive measure of value we then discuss budgetsetting and prioritization using multicriteria decision analysis issues not yet fully resolved next we discuss deliberative processes which represent another important approach for populationor planlevel decisions used by many health technology assessment bodies these use quantitative information on cea and other elements but the group decisions are reached by a deliberative voting process finally we briefly discuss the use of stated preference methods for developing hedonic value frameworks and conclude with some recommendations in this area 

Abstract_Summary:    The fifth section of our Special Task Force<br>report identifies and discusses two aggregation<br>issues: 1) aggregation of cost and benefit information<br>across individuals to a population level for benefit<br>plan decision making and 2) combining multiple<br>elements of value into a single value metric for<br>individuals. First, we argue that additional elements could<br>be included in measures of value, but such<br>elements have not generally been included in measures of<br>quality-adjusted lifeyears. For example, we describe a recently<br>developed extended costeffectiveness analysis (ECEA)<br>that provides a good example of how to use a broader<br>concept of utility. ECEA adds... 

-------------------------------------------


=========================================================
==========================Query==============================
=== Resources to support skilled nursing facilities and long term care facilities. =====
=========================================================
Score:    (Score: 0.7985) 

Paragraph:    health care workforce capacity building adequacy of health care workforce 

paper_id:   2e87670d524b1b76e2af6090d5cba472a6448f85 

Title:    Capacity building in health care professions<br>within the Gulf cooperation council countries:<br>paving the way forward 

Abstract:   background there is a worldwide shortage of health care workers this problem is particularly severe in the gulf cooperation council gcc countries because of shortages in certain medical disciplines due to a lack of nationallytrained professionals and a less developed educational system compared to other high income countries consequently gcc countries are heavily dependent on an expatriate health care workforce a problem exacerbated by high turnover we discuss challenges and potential strategies for improving and strengthening capacity building efforts in health care professions in the gcc main text in the gcc there are 139 schools providing professional health education in medicine dentistry pharmacy nursing midwifery and other specialties health education school density reported for the gcc countries ranges between 22 and 28 schools per one million inhabitants except in oman where it is 40 per one million inhabitants the gcc countries rely heavily on expatriate health professionals the number of physicians and nurses in the gcc countries are 21 and 45 per 1000 respectively compared to 28 and 79 among member countries of the organisation for economic cooperation and development oecd interestingly the number of dentists and pharmacists is higher in the gcc countries compared to oecd countries a nationally trained health care workforce is essential for the gcc countries physiotherapy and occupational therapy are two identified areas where growth and development are recommended customtailored continuing medical education and continuing professional development cpd programs can augment the skills of health practitioners and allow for the expansion of their scope of practice when warranted conclusion capacity building can play an essential role in addressing the major health challenges and improving the overall quality of health care in the region efforts aimed at increasing the number of locallytrained graduates and developing and implementing needbased cpd programs are vital for capacity building and lifelong learning in health care professions 

Abstract_Summary:    Background: There is a worldwide shortage of<br>health care workers. This problem is particularly<br>severe in the Gulf Cooperation Council (GCC)<br>countries because of shortages in certain medical<br>disciplines, due to a lack of nationallytrained<br>professionals and a less developed educational system<br>compared to other high income countries. Consequently,<br>GCC countries are heavily dependent on an<br>expatriate health care workforce; a problem exacerbated<br>by high turnover. We discuss challenges and<br>potential strategies for improving and strengthening<br>capacity building efforts in health care professions in<br>the GCC. Main text: In the GCC, there are 139 schools<br>providing professional health education in... 

-------------------------------------------
Score:    (Score: 0.7676) 

Paragraph:    with ili to an acute care facility 

paper_id:   67c79979c388cfcad87bb12d0ea644699500d736 

Title:    Guidance on the use of antiviral drugs for<br>influenza in acute care facilities in Canada, 2014–2015 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.7558) 

Paragraph:    key points that contribute to caregiver success are summarized in table 3  regular prescheduled followup the ability for vai initiated medical support and respite care for the ventilatorassisted individual or caregiver are especially important 

paper_id:   e20c3efee1c9653c22821eb34138d6138daefe40 

Title:    8(th) International conference on management<br>and rehabilitation of chronic respiratory<br>failure: the long summaries – Part 3 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.7511) 

Paragraph:    level of healthcare resources 

paper_id:   30fc9806c9b1ba0d786a31cad4a1d98b6642462d 

Title:    Pandemic influenza preparedness and health<br>systems challenges in Asia: results from rapid<br>analyses in 6 Asian countries 

Abstract:   background since 2003 asiapacific particularly southeast asia has received substantial attention because of the anticipation that it could be the epicentre of the next pandemic there has been active investment but earlier review of pandemic preparedness plans in the region reveals that the translation of these strategic plans into operational plans is still lacking in some countries particularly those with low resources the objective of this study is to understand the pandemic preparedness programmes the health systems context and challenges and constraints specific to the six asian countries namely cambodia indonesia lao pdr taiwan thailand and viet nam in the prepandemic phase before the start of h1n12009
the study relied on the systemic rapid assessment sysra toolkit which evaluates priority disease programmes by taking into account the programmes the general health system and the wider sociocultural and political context the components under review were external context stewardship and organisational arrangements financing resource generation and allocation healthcare provision and information systems qualitative and quantitative data were collected in the second half of 2008 based on a review of published data and interviews with key informants exploring past and current patterns of health programme and pandemic response
the study shows that health systems in the six countries varied in regard to the epidemiological context health care financing and health service provision patterns for pandemic preparation all six countries have developed national governance on pandemic preparedness as well as national pandemic influenza preparedness plans and avian and human influenza ahi response plans however the governance arrangements and the nature of the plans differed in the five developing countries the focus was on surveillance and rapid containment of poultry related transmission while preparation for later pandemic stages was limited the interfaces and linkages between health system contexts and pandemic preparedness programmes in these countries were explored
health system context influences how the six countries have been preparing themselves for a pandemic at the same time investment in pandemic preparation in the six asian countries has contributed to improvement in health system surveillance laboratory capacity monitoring and evaluation and public communications a number of suggestions for improvement were presented to strengthen the pandemic preparation and mitigation as well as to overcome some of the underlying health system constraints 

Abstract_Summary:    Background: Since 2003, Asia-Pacific,<br>particularly Southeast Asia, has received substantial<br>attention because of the anticipation that it could be the<br>epicentre of the next pandemic. There has been active<br>investment but earlier review of pandemic preparedness<br>plans in the region reveals that the translation of<br>these strategic plans into operational plans is<br>still lacking in some countries particularly those<br>with low resources. The objective of this study is to<br>understand the pandemic preparedness programmes, the<br>health systems context, and challenges and<br>constraints specific to the six Asian countries namely<br>Cambodia, Indonesia, Lao PDR, Taiwan, Thailand, and Viet<br>Nam in the... 

-------------------------------------------
Score:    (Score: 0.7450) 

Paragraph:    life support elements within the integrated management of pregnancy and childbirth eg enc 

paper_id:   737650f6ab6664da7582430d4b45ff9dd2e76866 

Title:    In-service training for health professionals<br>to improve care of seriously ill newborns and<br>children in low-income countries 

Abstract:   inservice training for health professionals to improve care of seriously ill newborns and children in lowincome countries review 

Abstract_Summary:    In-service training for health professionals<br>to improve care of seriously ill newborns and<br>children in low-income countries (Review) 

-------------------------------------------


=========================================================
==========================Query==============================
=== Mobilization of surge medical staff to address shortages in overwhelmed communities . =====
=========================================================
Score:    (Score: 0.8239) 

Paragraph:    immediate public health and medical support challenges included the identification triage and treatment of acutely sick and injured patients the management of chronic medical conditions in large number of evacuees with special health care needs the assessment communication and mitigation of public health risks mortuary support and the provision of assistance to state and local health officials to quickly reestablish health care deliver systems and public health infrastructures 

paper_id:   26e62203e2aecbd14310f48420b0f5b368453628 

Title:    Directions for Disaster Nursing Education in<br>the United States 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.7772) 

Paragraph:    many poor countries already have a health crisis and need massive international investments including mobilisation and strengthening of human resources to build sustainable health systems strong leadership and political commitment [73]  in the face of the pandemic threat primary health care in developing countries will need resources to develop a suite of policies including clarification of what essential primary health care will continue through a pandemic developing health workforce plans that may entail diverting clinicians from other areas of the health workforce establishing nonhierarchical links between primary health care hospitals and public health and injecting funds into hospital and primary care preparedness simultaneously 

paper_id:   646484c8081b3f1bf336bae89411c2aa86885788 

Title:    General Practice and Pandemic Influenza: A<br>Framework for Planning and Comparison of Plans in Five<br>Countries 

Abstract:   background although primary health care and in particular general practice will be at the frontline in the response to pandemic influenza there are no frameworks to guide systematic planning for this task or to appraise available plans for their relevance to general practice we aimed to develop a framework that will facilitate planning for general practice and used it to appraise pandemic plans from australia england usa new zealand and canada
methodologyprincipal findings we adapted the haddon matrix to develop the framework populating its cells through a multimethod study that incorporated the peerreviewed and grey literature interviews with general practitioners practice nurses and senior decisionmakers and desktop simulation exercises we used the framework to analyse 89 publiclyavailable jurisdictional plans at similar managerial levels in the five countries the framework identifies four functional domains clinical care for influenza and other needs public health responsibilities the internal environment and the macroenvironment of general practice no plan addressed all four domains most plans either ignored or were sketchy about noninfluenza clinical needs and about the contribution of general practice to public health beyond surveillance collaborations between general practices were addressed in few plans and interrelationships with the broader health system even less frequently
conclusions this is the first study to provide a framework to guide general practice planning for pandemic influenza the framework helped identify critical shortcomings in available plans engaging general practice effectively in planning is challenging particularly where governance structures for primary health care are weak we identify implications for practice and for research 

Abstract_Summary:    Background: Although primary health care, and<br>in particular, general practice will be at the<br>frontline in the response to pandemic influenza, there<br>are no frameworks to guide systematic planning for<br>this task or to appraise available plans for their<br>relevance to general practice. We aimed to develop a<br>framework that will facilitate planning for general<br>practice, and used it to appraise pandemic plans from<br>Australia, England, USA, New Zealand and<br>Canada.
Methodology/Principal Findings: We adapted the Haddon matrix to<br>develop the framework, populating its cells through a<br>multi-method study that incorporated the peer-reviewed and<br>grey literature, interviews with general<br>practitioners, practice... 

-------------------------------------------
Score:    (Score: 0.7747) 

Paragraph:    finally in some countries particularly the smaller ones substantial shortfalls in preparedness were revealed concerning accommodating a surge of cases in healthcare facilities testing for multiple cases and contacts and mobilizing staff for contact tracing countries need to be prepared for a scenario that rapidly overwhelms the capacity of health authorities they should therefore consider detailed surge capacity planning that includes standby arrangements with other ministries eg defence or interior and civil society or international partners 

paper_id:   814ecf9043112382eaae1310c6359ddc70557bba 

Title:    Assessment of Ebola virus disease<br>preparedness in the WHO South-East Asia Region 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------
Score:    (Score: 0.7723) 

Paragraph:    in contemporary emergency care triage is regarded as an essential function not only during massive influx of patients as in disasters epidemics and pandemics but also in regular emergency care departments the burden in emergency care is increasing and so are the expectations of patients [1]  in hospitals that apply triage for regular emergency care triage is the first point of contact with the ed assessment by the triage officers involves a combination of the chief complaint of the patient general appearance and at times recording of vital signs [25] 

paper_id:   34190b8df3ed825e3c49e41a3782d04acf1e21e9 

Title:    Emergency department triage: an ethical<br>analysis 

Abstract:   background emergency departments across the globe follow a triage system in order to cope with overcrowding the intention behind triage is to improve the emergency care and to prioritize cases in terms of clinical urgency
discussion in emergency department triage medical care might lead to adverse consequences like delay in providing care compromise in privacy and confidentiality poor physicianpatient communication failing to provide the necessary care altogether or even having to decide whose life to save when not everyone can be saved these consequences challenge the ethical quality of emergency care this article provides an ethical analysis of routine emergency department triage the four principles of biomedical ethics viz respect for autonomy beneficence nonmaleficence and justice provide the starting point and help us to identify the ethical challenges of emergency department triage however they do not offer a comprehensive ethical view to address the ethical issues of emergency department triage from a more comprehensive ethical view the care ethics perspective offers additional insights summary we integrate the results from the analysis using four principles of biomedical ethics into care ethics perspective on triage and propose an integrated clinically and ethically based framework of emergency department triage planning as seen from a comprehensive ethics perspective that incorporates both the principlesbased and careoriented approach 

Abstract_Summary:    Background: Emergency departments across the<br>globe follow a triage system in order to cope with<br>overcrowding. The intention behind triage is to improve the<br>emergency care and to prioritize cases in terms of<br>clinical urgency.
Discussion: In emergency<br>department triage, medical care might lead to adverse<br>consequences like delay in providing care, compromise in<br>privacy and confidentiality, poor physician-patient<br>communication, failing to provide the necessary care<br>altogether, or even having to decide whose life to save when<br>not everyone can be saved. These consequences<br>challenge the ethical quality of emergency care. This<br>article provides an ethical analysis of "routine"<br>emergency department... 

-------------------------------------------
Score:    (Score: 0.7684) 

Paragraph:    the fi fth priority is emergency preparedness and response protracted emergencies seem almost to have become a way of life in some parts of the region and more than half of the countries are currently facing either acute or chronic crises the major source of emergencies is civil unrest and violent confl ict the consequences are clear in the expanding humanitarian crisis in syria and its neighbours with rising numbers of people displaced health systems in all countries aff ected are facing major diffi culties in coping with the demands collective action and solidarity are needed to deliver health services to refugees and host communities and to increase the resilience of countries to emergencies and ensure eff ective public health responses during crises 

paper_id:   6c7de5a96730bd3962f86713db5aebdb4724f14a 

Title:    Responding to priority health challenges in<br>the Arab world 

Abstract:   nan 

Abstract_Summary:   Not provided. 

-------------------------------------------


=========================================================
==========================Query==============================
=== Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies . =====
=========================================================
Score:    (Score: 0.8562) 

Paragraph:    patients were categorized according to their comorbidity status ie patients without comorbidity with chronic respiratory disease or with other comorbidities and compared using chisquared tests viral and bacterial infection status were compared by age group and comorbidity status using chisquared tests fishers exact tests and chisquared tests for trend inhospital mortality rates were calculated by viral and bacterial infection status and compared with those of the virusnegative group the effects of viral infection on inhospital mortality were expressed as risk ratios with 95 confidence intervals ci and estimated using poisson regression models with robust standard errors age study site comorbidity status duration of symptoms month of diagnosis antibiotic use and presence of bacteria were considered potential confounders based on prior knowledge and were included in the multiple regression models for patients whose onset of symptoms were unknown 5 we coded those missing values as unknown and included all patients in our analysis the data were analysed using stata version 13 stata corp college station tx usa 

paper_id:   6302e80258b85ac4f27c6db06bcf9e245e981ba9 

Title:    The impact of virus infections on pneumonia<br>mortality is complex in adults: a prospective<br>multicentre observational study 

Abstract:   background various viruses are known to be associated with pneumonia however the impact of viral infections on adult pneumonia mortality remains unclear this study aimed to clarify the effect of virus infection on pneumonia mortality among adults stratified by virus type and patient comorbidities
methods this multicentre prospective study enrolled pneumonia patients aged 15 years from september 2011 to august 2014 sputum samples were tested by inhouse multiplex polymerase chain reaction assays to identify 13 respiratory viruses viral infection status and its effect on inhospital mortality were examined by age group and comorbidity status results a total of 2617 patients were enrolled in the study and 778 was aged 65 years 574 219 did not have comorbidities 790 302 had chronic respiratory disease and 1253 479 had other comorbidities viruses were detected in 605 231 patients human rhinovirus 98 was the most frequently identified virus followed by influenza a 39 and respiratory syncytial virus 39 respiratory syncytial virus was more frequently identified in patients with chronic respiratory disease 47 than those with other comorbidities 42 and without comorbidities 21 p  0037 the frequencies of other viruses were almost identical between the three groups virus detection overall was not associated with increased mortality adjusted risk ratio arr 076 95 ci 053109 however influenza virus a and b were associated with threefold higher mortality in patients with chronic respiratory disease but not with other comorbidities arr 338 95 ci 154742 intriguingly paramyxoviruses were associated with dramatically lower mortality in patients with other comorbidities arr 010 95 ci 001070 but not with chronic respiratory disease these effects were not affected by age group conclusions the impact of virus infections on pneumonia mortality varies by virus type and comorbidity status in adults 

Abstract_Summary:    Background: Various viruses are known to be<br>associated with pneumonia. However, the impact of viral<br>infections on adult pneumonia mortality remains unclear.<br>This study aimed to clarify the effect of virus<br>infection on pneumonia mortality among adults<br>stratified by virus type and patient<br>comorbidities.
Methods: This multicentre prospective study enrolled<br>pneumonia patients aged ≥15 years from September 2011 to<br>August 2014. Sputum samples were tested by in-house<br>multiplex polymerase chain reaction assays to identify<br>13 respiratory viruses. Viral infection status<br>and its effect on in-hospital mortality were<br>examined by age group and comorbidity status. Results: A<br>total of 2617 patients were... 

-------------------------------------------
Score:    (Score: 0.8340) 

Paragraph:    statistical analysis associations between categorical and continuous variables were analyzed using the mannwhitneyu test or kruskalwallis test for continuous variables and the fisher exact test for dichotomous variables spearmans rank correlation coefficient was used to assess a general monotonic trend between two continuous variables a simple linear regression model was used to measure linear associations between rsv viral load and age or day of illness or number of leucocytes in blood multivariable logistic regression analysis was performed to assess risk factors for severe disease or long hospitalization the following variables were considered for the models age sex premature birth previous hospitalization for respiratory illness other household members sick at home living with smokers number of members at home fever rsv infection rsv positivity viral load and subgroup and rv infection rv positivity as single infection or coinfection with rsv the models predictive ability was investigated by calculating the area under the receiver operating characteristic roc curve of the model auc ie the higher the auc the better prediction power the model has the hosmerlemeshow goodnessoffit test was done to assess model adequacy the hosmerlemeshow test of goodnessoffit tests the null hypothesis that there is no difference between the observed and predicted values of the response variables therefore when the test is not significant p005 the null hypothesis cannot be rejected and this means that the model fits the data well risk factors for disease severity or long hospitalization were further assessed using odds ratios or and 95 confidence intervals 95 cis all statistical tests were performed as twotailed tests at 5 significance in either r version 2131 r foundation for statistical computing vienna austria or intercooled stata version 92 college station tx usa 

paper_id:   ebf7e1110aa56a3066f9590befe475eb12b3eaa1 

Title:    Respiratory Syncytial Virus and Other Viral<br>Infections among Children under Two Years Old in Southern<br>Vietnam 2009-2010: Clinical Characteristics and<br>Disease Severity 

Abstract:   despite a high burden of respiratory syncytial virus rsv infections among children data on demographic and clinical characteristics of rsv are scarce in low and middle income countries this study aims to describe the viral etiologies the demographic epidemiological and clinical characteristics of children under two years of age who were hospitalized with a lower respiratory tract infections lrti focusing on rsv prevalence seasonality subgroups viral load and its association with disease severity
a prospective study among children under two years of age hospitalized with lrti was conducted in two referral pediatric hospitals in ho chi minh city vietnam from may 2009 to december 2010 sociodemographic clinical data and nasopharyngeal swabs were collected on enrolment and discharge multiplex realtime rtpcr 13 viruses and quantitative rsv rtpcr were used to identify viral pathogens rsv load and subgroups
among 632 cases 48 were rsv positive rsv infections occurred at younger age than three other leading viral infections ie rhinovirus rv metapneumovirus mpv parainfluenza virus piv3 and were significantly more frequent in the first 6 months of life clinical severity score of rsv infection was significantly higher than piv3 but not for rv or mpv in multivariate analysis rv infection was significantly associated with severity while rsv infection was not among rsv infections neither viral load nor viral coinfections were significantly associated with severity young age and having fever at admission were significantly associated with both rsv and lrti severity a shift in rsv subgroup predominance was observed during two consecutive rainy seasons but was not associated with severity
we report etiologies the epidemiological and clinical characteristics of lrti among hospitalized children under two years of age and risk factors of rsv and lrti severity
sociodemographic data medical history clinical data from enrolment to discharge were recorded in standardized case report forms crfs nasopharyngeal swabs and edta blood respiratory syncytial virus and other viral respiratory infections in children under two years old plos one  

Abstract_Summary:    Despite a high burden of respiratory syncytial<br>virus (RSV) infections among children, data on<br>demographic and clinical characteristics of RSV are scarce<br>in low and middle income countries. This study<br>aims to describe the viral etiologies, the<br>demographic, epidemiological, and clinical<br>characteristics of children under two years of age who were<br>hospitalized with a lower respiratory tract infections<br>(LRTI), focusing on RSV (prevalence, seasonality,<br>subgroups, viral load) and its association with disease<br>severity.
A prospective study among children under two<br>years of age, hospitalized with LRTI was conducted in<br>two referral pediatric hospitals in Ho Chi Minh<br>City, Vietnam, from May... 

-------------------------------------------
Score:    (Score: 0.8151) 

Paragraph:    stochastic and deterministic sirvtype epidemic models in which individuals are susceptible s infected and infectious i 1 or i 2  recovered and immune r or partially protected against infection by antiviral prophylactic treatment v form the basis of the analyses figure 1 shows a schematic of the model details are given in text s1 throughout susceptible individuals enter the population by birth the natural death rate of susceptible and recovered individuals is denoted by m and the excess mortality while on antiviral treatment is given by c hence life expectancy is m 21 in the absence of infection and antiviral control and mc 21 while on antiviral drugs in the infected classes i 1 and i 2  the excess death rates resulting from infection are given by n and n12ave i  where ave i is the antiviral efficacy for infectiousness and virulence in the following c and n will be referred to succinctly as the cost of antiviral prophylaxis and infection from the susceptible class individuals move to the protected and infected classes at rates s and l the parameter l is colloquially called the force of infection and it depends on the prevalence of infection text s1 individuals in class v are infected at a reduced rate l12ave s  where ave s denotes the antiviral efficacy for susceptibility this implies that individuals in class v cannot be infected at all if ave s  1 while the antiviral drug provides no protection against infection if ave s  0 finally the rates of recovery and noncompliance are given by a and r respectively an overview of the model parameters and their default values is given in table 1  details of the model analyses are provided in text s1 

paper_id:   26670350d15f4a1f0a7072ad985d98b21c28fd5a 

Title:    Self-Interest versus Group-Interest in<br>Antiviral Control 

Abstract:   antiviral agents have been hailed to hold considerable promise for the treatment and prevention of emerging viral diseases like h5n1 avian influenza and sars however antiviral drugs are not completely harmless and the conditions under which individuals are willing to participate in a largescale antiviral drug treatment program are as yet unknown we provide population dynamical and game theoretical analyses of largescale prophylactic antiviral treatment programs throughout we compare the antiviral control strategy that is optimal from the public health perspective with the control strategy that would evolve if individuals make their own rational decisions to this end we investigate the conditions under which a largescale antiviral control program can prevent an epidemic and we analyze at what point in an unfolding epidemic the risk of infection starts to outweigh the cost of antiviral treatment this enables investigation of how the optimal control strategy is moulded by the efficacy of antiviral drugs the risk of mortality by antiviral prophylaxis and the transmissibility of the pathogen our analyses show that there can be a strong incentive for an individual to take less antiviral drugs than is optimal from the public health perspective in particular when public health asks for early and aggressive control to prevent or curb an emerging pathogen for the individual antiviral drug treatment is attractive only when the risk of infection has become nonnegligible it is even possible that from a public health perspective a situation in which everybody takes antiviral drugs is optimal while the process of individual choice leads to a situation where nobody is willing to take antiviral drugs 

Abstract_Summary:    Antiviral agents have been hailed to hold<br>considerable promise for the treatment and prevention of<br>emerging viral diseases like H5N1 avian influenza and<br>SARS. However, antiviral drugs are not completely<br>harmless, and the conditions under which individuals are<br>willing to participate in a large-scale antiviral drug<br>treatment program are as yet unknown. We provide<br>population dynamical and game theoretical analyses of<br>large-scale prophylactic antiviral treatment programs.<br>Throughout we compare the antiviral control strategy that<br>is optimal from the public health perspective<br>with the control strategy that would evolve if<br>individuals make their own, rational decisions. To this end<br>we investigate... 

-------------------------------------------
Score:    (Score: 0.8138) 

Paragraph:    when substantial heterogeneity was found i 2 40 a priori defined subgroup analyses were performed which included age  18 years versus 18 years the nature of the underlying respiratory illness aris bronchiolitis radiologically confirmed community acquired pneumonia cap respectively the presence of underlying comorbidities the impact of overall viral infections versus specific viral pathogens and high versus low risk of bias in the included studies as defined above interaction tests for subgroup differences were conducted using the chisquare and i 2 statistic subgroup credibility was examined using the criteria described by sun and colleagues [13]  we made a posthoc stratified analysis of children by age subgroups based on data availability in the abstracted studies infants 023 months of age preschool children 059 months of age and children 017 years old 

paper_id:   a65d8f4193a3b03c5d20fac1972e0756ecf2d328 

Title:    Clinical Disease Severity of Respiratory<br>Viral Co-Infection versus Single Viral Infection: A<br>Systematic Review and Meta-Analysis 

Abstract:   results from cohort studies evaluating the severity of respiratory viral coinfections are conflicting we conducted a systematic review and metaanalysis to assess the clinical severity of viral coinfections as compared to single viral respiratory infections
we searched electronic databases and other sources for studies published up to january 28 2013 we included observational studies on inpatients with respiratory illnesses comparing the clinical severity of viral coinfections to single viral infections as detected by molecular assays the primary outcome reflecting clinical disease severity was length of hospital stay los a randomeffects model was used to conduct the metaanalyses
results twentyone studies involving 4280 patients were included the overall quality of evidence applying the grade approach ranged from moderate for oxygen requirements to low for all other outcomes no significant differences in length of hospital stay los mean difference md 2020 days 95 ci 2094 053 p  059 or mortality rr 244 95 ci 086 691 p  009 were documented in subjects with viral coinfections compared to those with a single viral infection there was no evidence for differences in effects across age subgroups in post hoc analyses with the exception of the higher mortality in preschool children rr 982 95 ci 309 3120 p0001 with viral coinfection as compared to other age groups i 2 for subgroup analysis 64 p  004
conclusions no differences in clinical disease severity between viral coinfections and single respiratory infections were documented the suggested increased risk of mortality observed amongst children with viral coinfections requires further investigation 

Abstract_Summary:    Results from cohort studies evaluating the<br>severity of respiratory viral co-infections are<br>conflicting. We conducted a systematic review and<br>meta-analysis to assess the clinical severity of viral<br>co-infections as compared to single viral respiratory<br>infections.
We searched electronic databases and other<br>sources for studies published up to January 28, 2013. We<br>included observational studies on inpatients with<br>respiratory illnesses comparing the clinical severity of<br>viral co-infections to single viral infections as<br>detected by molecular assays. The primary outcome<br>reflecting clinical disease severity was length of<br>hospital stay (LOS). A random-effects model was used to<br>conduct the meta-analyses.
Results: Twenty-one<br>studies involving 4,280... 

-------------------------------------------
Score:    (Score: 0.8112) 

Paragraph:    multivariable logistic regression was used to estimate the odds ratio or and 95 confidence interval ci associated with hospitalization outcomes between those infected with rsv and those infected with influenza after adjustment for potential differences between cohorts including age sex raceethnicity history of influenza and pneumococcal vaccination prior chronic obstructive pulmonary disease copdchronic bronchitisemphysema charlson comorbidity index [17]  history of healthcare utilization and recent use of antivirals antibiotics or steroids oneyear survival after admission was estimated using the kaplanmeier method and survival rates between hospitalized persons infected with rsv or influenza were compared using the logrank test the analyses were performed using sas enterprise guide 51 sas institute inc cary nc 

paper_id:   4261cfd373572b30e170171afce095e6eb5d806a 

Title:    Severe Morbidity and Mortality Associated<br>With Respiratory Syncytial Virus Versus Influenza<br>Infection in Hospitalized Older Adults 

Abstract:   background respiratory syncytial virus rsv is an important cause of serious respiratory illness in older adults comparison of rsv and influenza infection in hospitalized older adults may increase awareness of adult rsv disease burden
methods hospitalized adults aged 60 years who tested positive for rsv or influenza between 1 january 2011 and 30 june 2015 were identified from kaiser permanente southern california electronic medical records baseline characteristics comorbidities utilization and outcomes were compared
results the study included 645 rsvand 1878 influenzainfected hospitalized adults patients with rsv were older than those with influenza mean 785 vs 774 years p  035 and more likely to have congestive heart failure 353 vs 245 p  001 and chronic obstructive pulmonary disease copd 298 vs 243 p  006 at baseline in adjusted analyses rsv infection was associated with greater odds of length of stay 7 days odds ratio [or]  15 95 confidence interval [ci] 1218 p  001 pneumonia or  27 95 ci 2232 p  001 intensive care unit admission or  13 95 ci 1017 p  023 exacerbation of copd or  17 95 ci 1324 p  001 and greater mortality within 1 year of admission or  13 95 ci 1016 p  019
conclusions rsv infection may result in greater morbidity and mortality among older hospitalized adults than influenza increased recognition of adult rsv disease burden will be important in the evaluation and use of new rsv vaccines and antivirals 

Abstract_Summary:    Background. Respiratory syncytial virus<br>(RSV) is an important cause of serious respiratory<br>illness in older adults. Comparison of RSV and<br>influenza infection in hospitalized older adults may<br>increase awareness of adult RSV disease<br>burden.
Methods. Hospitalized adults aged ≥60 years who tested<br>positive for RSV or influenza between 1 January 2011 and<br>30 June 2015 were identified from Kaiser<br>Permanente Southern California electronic medical<br>records. Baseline characteristics, comorbidities,<br>utilization, and outcomes were compared.
Results. The<br>study included 645 RSV-and 1878 influenza-infected<br>hospitalized adults. Patients with RSV were older than those<br>with influenza (mean, 78.5 vs 77.4 years; P = .035)<br>and more... 

-------------------------------------------

In [0]:
#import pickle as pkl
#with open("/content/drive/My Drive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "wb") as file_:
#  pkl.dump(corpus_embeddings,file_)